<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-13 15:19:32
-------------------
qualified stocks: 91
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  3.50 L
Current:  1.33 C
-------------------
Today PnL: -1.10 L (-0.82%)
Current PnL: -16.67 L (-11.62%)
CY Booked + Current PnL: -10.30 L (-7.18%)
-------------------
Total profit:  2.89 L
Total loss:  -19.56 L
-------------------
Total Booked + Current PnL: 16.68 L (14.31%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.78%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 72.57 L (54.44%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,827.06,983.0,-5.02,H-LC,86.81,96635.0,11448.0,4619.0,-0.58,13.44,4.78,18.85,35.0,2.48,0.74,28.66,XY25,NTT,INSURANCE
60,RELIANCE,1291.83,1526.0,4.61,X-LC,31.87,155674.0,14865.0,10664.0,-0.93,10.56,6.85,18.13,17.0,1.39,1.20,22.92,XY25,NTT,REFINERIES
82,UJJIVANSFB,52.77,53.0,52.86,M-SC,78.02,132192.0,-10287.0,10906.0,1.14,-7.22,8.25,0.44,242.0,-0.94,1.02,56.02,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,-0.40,M-LC,43.96,99968.0,21606.0,12426.0,-2.14,27.57,12.43,43.43,77.0,1.74,0.77,34.36,XY24,NTT,MISC
76,TATAELXSI,7332.28,7332.0,-14.64,X-MC,69.23,89572.0,-13080.0,13078.0,-1.17,-12.74,14.60,-0.00,57.0,-1.00,0.69,36.16,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAIL,130.64,228.00,96.05,M-MC,79.12,222913.0,-2049.0,169704.0,-0.40,-0.91,76.13,74.53,184.0,-0.01,1.72,29.42,XY24,BTT,STEEL
32,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,62.64,151624.0,711.0,59740.0,-0.70,0.47,39.40,40.06,100.0,0.01,1.17,24.02,AR,ATH,AUTO
88,VOLTAS,1278.28,1929.20,-29.76,H-MC,39.56,191535.0,-207.0,97836.0,-1.01,-0.11,51.08,50.92,99.0,-0.00,1.48,6.09,XY25,ATH,AC
56,PGHH,13388.00,18106.42,-20.14,X-MC,12.09,201510.0,690.0,70085.0,0.25,0.34,34.78,35.24,53.0,0.01,1.55,5.04,X40,ATH,FMCG
19,CIPLA,1495.00,1795.00,-17.32,H-LC,21.98,100922.0,757.0,19347.0,0.23,0.76,19.17,20.07,29.0,0.04,0.78,8.78,X40N,BTT,PHARMA


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7450.00,M-SC,100.00,82850.0,-10960.0,87581.0,3.06,-11.68,105.71,81.67,238.0,-0.13,0.64,60.44,XR,NTT,CERAMICS
62,ROUTE,1558.40,2249.26,-33.00,M-SC,90.11,72267.0,-30587.0,76184.0,1.61,-29.74,105.42,44.33,188.0,-0.40,0.56,20.01,SR,ATH,IT
61,REPCOHOME,515.07,880.00,-53.80,H-SC,50.55,169429.0,-35569.0,180815.0,1.54,-17.35,106.72,70.85,134.0,-0.20,1.31,36.77,XY24,NTT,FINANCE
82,UJJIVANSFB,52.77,53.00,52.86,M-SC,78.02,132192.0,-10287.0,10906.0,1.14,-7.22,8.25,0.44,242.0,-0.94,1.02,56.02,OX40N,NTT,BANKS
57,QUESS,711.66,986.00,NaN,NaN,1.10,68416.0,-87438.0,147519.0,0.87,-56.10,215.62,38.55,191.0,-0.59,0.53,4.76,XY24,NTT,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VALIANTORG,512.64,838.0,-385.09,H-SC,98.90,121225.0,-12061.0,96653.0,-4.94,-9.05,79.73,63.47,142.0,-0.12,0.93,103.20,XR,NTT,CHEMICALS
58,RAJESHEXPO,517.65,518.0,1777.53,M-SC,27.47,51214.0,-85963.0,86055.0,-3.67,-62.67,168.03,0.07,232.0,-1.00,0.39,27.37,OX40N,NTT,JEWELLERY
87,VIPIND,488.80,489.0,-927.68,H-SC,91.21,78809.0,-16507.0,16542.0,-3.02,-17.32,20.99,0.04,153.0,-1.00,0.61,55.44,OX40N,NTT,MISC
59,RELAXO,902.64,1176.0,-40.36,H-SC,25.27,53907.0,-61631.0,96623.0,-2.97,-53.34,179.24,30.28,135.0,-0.64,0.42,5.31,X40N,NTT,FOOTWEAR
47,KANSAINER,299.63,340.0,-68.77,H-SC,7.69,218250.0,-51417.0,87758.0,-2.94,-19.07,40.21,13.47,144.0,-0.59,1.68,9.24,XY24,NTT,PAINTS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UJJIVANSFB,52.77,53.0,52.86,M-SC,78.02,132192.0,-10287.0,10906.0,1.14,-7.22,8.25,0.44,242.0,-0.94,1.02,56.02,OX40N,NTT,BANKS
76,TATAELXSI,7332.28,7332.0,-14.64,X-MC,69.23,89572.0,-13080.0,13078.0,-1.17,-12.74,14.60,-0.00,57.0,-1.00,0.69,36.16,OX40N,NTT,IT
74,SYMPHONY,1306.42,1306.0,-29.94,M-SC,2.20,149130.0,-22011.0,21952.0,-2.17,-12.86,14.72,-0.03,190.0,-1.00,1.15,7.18,OX40N,NTT,DURABLES
18,CERA,8421.60,8422.0,-11.69,X-SC,68.13,91130.0,-18351.0,18354.0,0.06,-16.76,20.14,0.00,65.0,-1.00,0.70,37.93,OX40N,NTT,CERAMICS
87,VIPIND,488.80,489.0,-927.68,H-SC,91.21,78809.0,-16507.0,16542.0,-3.02,-17.32,20.99,0.04,153.0,-1.00,0.61,55.44,OX40N,NTT,MISC


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-10.62,X-LC,46.15,286758.0,25157.0,120467.0,-0.41,9.62,42.01,55.67,1.0,0.21,2.21,16.31,X40,BTT,IT
78,TCS,3794.03,4998.00,-17.44,X-LC,19.78,268975.0,-26959.0,120877.0,0.41,-9.11,44.94,31.73,2.0,-0.22,2.07,7.66,X40,BTT,IT
34,HINDUNILVR,2413.81,2723.00,-17.43,X-LC,16.48,259627.0,-10720.0,45357.0,-0.63,-3.97,17.47,12.81,5.0,-0.24,2.00,7.45,XY25,NTT,FMCG
14,BRITANNIA,4983.33,6446.05,11.96,X-LC,41.76,256611.0,27378.0,39903.0,0.16,11.94,15.55,29.35,7.0,0.69,1.98,21.93,XY25,ATH,FMCG
55,NESTLEIND,2268.60,2755.00,-12.47,X-LC,28.57,277980.0,12554.0,44366.0,-0.41,4.73,15.96,21.44,10.0,0.28,2.14,11.22,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-0.4,M-LC,43.96,99968.0,21606.0,12426.0,-2.14,27.57,12.43,43.43,77.0,1.74,0.77,34.36,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,43.96,99968.0,21606.0,12426.0,-2.14,27.57,12.43,43.43,77.0,1.74,0.77,34.36,XY24,NTT,MISC
32,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,62.64,151624.0,711.0,59740.0,-0.70,0.47,39.40,40.06,100.0,0.01,1.17,24.02,AR,ATH,AUTO
17,CAMS,3643.00,5250.99,-0.89,H-SC,71.43,111496.0,9492.0,35534.0,-2.23,9.31,31.87,44.14,125.0,0.27,0.86,27.06,X40N,ATH,MISC
83,UNIONBANK,123.87,163.00,-4.91,M-LC,82.42,166411.0,25571.0,18921.0,-2.75,18.16,11.37,31.59,86.0,1.35,1.28,44.55,XY24,NTT,BANKS
90,WIPRO,243.46,420.00,-5.63,M-LC,45.05,161343.0,10398.0,99065.0,0.24,6.89,61.40,72.51,68.0,0.10,1.24,11.16,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,WHIRLPOOL,1167.49,2270.00,-33.02,M-SC,53.85,195216.0,23595.0,138467.0,-0.90,13.75,70.93,94.43,194.0,0.17,1.50,44.08,XR,NTT,DURABLES
90,WIPRO,243.46,420.00,-5.63,M-LC,45.05,161343.0,10398.0,99065.0,0.24,6.89,61.40,72.51,68.0,0.10,1.24,11.16,XR,NTT,IT
11,BANKINDIA,116.91,190.00,-29.44,M-MC,64.84,186559.0,6751.0,105667.0,-1.20,3.75,56.64,62.52,170.0,0.06,1.44,34.26,XR,NTT,BANKS
39,INDIAMART,2327.09,4871.06,-49.79,H-SC,56.04,130899.0,7563.0,127260.0,-0.96,6.13,97.22,109.32,122.0,0.06,1.01,29.32,AR,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,62.64,151624.0,711.0,59740.0,-0.70,0.47,39.40,40.06,100.0,0.01,1.17,24.02,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.0,-53.72,L-SC,96.70,68582.0,-44967.0,85014.0,-0.17,-39.60,123.96,35.27,260.0,-0.53,0.53,67.86,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.0,7450.00,M-SC,100.00,82850.0,-10960.0,87581.0,3.06,-11.68,105.71,81.67,238.0,-0.13,0.64,60.44,XR,NTT,CERAMICS
52,MASFIN,326.60,399.5,-23.22,H-SC,70.33,87255.0,-10725.0,32598.0,-1.09,-10.95,37.36,22.32,141.0,-0.33,0.67,26.55,XR,ATH,FINANCE
50,LICI,827.06,983.0,-5.02,H-LC,86.81,96635.0,11448.0,4619.0,-0.58,13.44,4.78,18.85,35.0,2.48,0.74,28.66,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.0,-0.40,M-LC,43.96,99968.0,21606.0,12426.0,-2.14,27.57,12.43,43.43,77.0,1.74,0.77,34.36,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.0,7450.00,M-SC,100.0,82850.0,-10960.0,87581.0,3.06,-11.68,105.71,81.67,238.0,-0.13,0.64,60.44,XR,NTT,CERAMICS
85,VALIANTORG,512.64,838.0,-385.09,H-SC,98.9,121225.0,-12061.0,96653.0,-4.94,-9.05,79.73,63.47,142.0,-0.12,0.93,103.20,XR,NTT,CHEMICALS
46,JPPOWER,18.73,26.2,-33.95,L-SC,97.8,133891.0,-7951.0,64522.0,-0.28,-5.61,48.19,39.88,258.0,-0.12,1.03,35.69,XY24,NTT,POWER
21,COFFEEDAY,59.14,80.0,-53.72,L-SC,96.7,68582.0,-44967.0,85014.0,-0.17,-39.60,123.96,35.27,260.0,-0.53,0.53,67.86,XR,NTT,HOTELS
79,TITAGARH,1097.23,1548.0,-10.53,H-SC,95.6,157342.0,-34673.0,113554.0,-1.62,-18.06,72.17,41.08,151.0,-0.31,1.21,33.17,XY24,NTT,ENGINEERING


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.78
1,25,43.25
2,50,72.01


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.94
XY25     14.30
X40      12.60
XR       11.02
OX40N     8.51
X40N      8.48
AR        8.13
X5K       2.34
X200      1.51
SR        1.22
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.65
X-LC    15.61
M-SC    14.40
H-LC    11.13
H-MC     9.03
M-LC     8.50
X-MC     7.71
M-MC     5.81
X-SC     2.83
L-SC     2.49
L-LC     1.12
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.67,-2.90,37.33
IT,11.47,-11.63,68.31
BANKS,8.54,-10.00,54.34
MISC,6.61,-26.26,75.64
FINANCE,6.39,-21.23,67.88
PAINTS,5.66,-18.74,42.15
ELECTRICAL,5.16,-4.84,50.31
HEALTHCARE,4.78,-3.87,34.22
AUTO,4.70,-11.02,59.84


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2787963.0,25
XR,1129767.0,13
AR,1016333.0,9
X40,684194.0,9
XY25,534052.0,10
X40N,430005.0,8
OX40N,325959.0,11
SR,186932.0,2
X5K,97784.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1947631.0,19
M-SC,1458924.0,18
X-LC,638375.0,9
M-MC,591559.0,5
H-MC,555778.0,7
H-LC,510722.0,10
X-MC,416372.0,6
M-LC,341648.0,6
L-SC,274744.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,915838.0,6
M-SC,XY24,587501.0,5
H-SC,AR,520044.0,3
M-MC,XY24,397761.0,3
M-SC,XR,343732.0,4
X-LC,X40,323821.0,3
H-MC,XY24,224922.0,2
M-LC,XY24,223816.0,4
H-SC,XR,205191.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
